# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import argparse
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.swin_transformer_1d as module_arch_swin_transformer
import model.beat_aligned_transformer as module_arch_beat_aligned_transformer

In [3]:
from parse_config import ConfigParser
from trainer import Trainer_beat_aligned_data
# from evaluater import Evaluater_beat_aligned_data
from evaluater import Evaluater_beat_aligned_data_CODE as Evaluater_beat_aligned_data
# from model.metric import ChallengeMetric
from model.metric import CODEMetric as ChallengeMetric
from utils.util import load_model
from utils.lr_scheduler import CosineAnnealingWarmUpRestarts, GradualWarmupScheduler
import datetime

In [4]:
files_models = {
    "swin_transformer_1d": ['swin_transformer'],
    "beat_aligned_transformer": ['beat_aligned_transformer'],
}

In [5]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str,
                    help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default='0', type=str,
                    help='indices of GPUs to enable (default: all)')
args.add_argument('-s', '--seed', type=int, default=0)
args.add_argument('-t', '--only_test', default=False, type=bool,
                    help='only test (default: False)')

_StoreAction(option_strings=['-t', '--only_test'], dest='only_test', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, help='only test (default: False)', metavar=None)

In [6]:
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]

In [7]:
config = ConfigParser.for_notebook(args, options)

In [8]:
import os
print("torch.cuda.device_count(): ", torch.cuda.device_count())
print("CUDA_VISIBLE_DEVICES: ", os.environ["CUDA_VISIBLE_DEVICES"])

torch.cuda.device_count():  1
CUDA_VISIBLE_DEVICES:  0


# main

In [9]:
logger = config.get_logger('train')

# build model architecture, then print to console
global model
for file, types in files_models.items():
    for type in types:
        if config["arch"]["type"] == type:
            model = config.init_obj('arch', eval("module_arch_" + file))
            logger.info(model)
            if config['arch'].get('weight_path', False):
                model = load_model(model, config["arch"]["weight_path"])

beat_aligned_transformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(8, 95, kernel_size=(1, 5), stride=(1, 5))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=96, input_resolution=80, depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=96, input_resolution=80, num_heads=8, window_size=5, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            dim=96, window_size=(5, 5), num_heads=8
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(


In [10]:
criterion = getattr(module_loss, config['loss']['type'])
challenge_metrics = ChallengeMetric(config['data_loader']['args']['label_dir'])
metrics = [getattr(challenge_metrics, met) for met in config['metrics']]

In [11]:
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = config.init_obj('optimizer', torch.optim, trainable_params)

In [12]:
if config["lr_scheduler"]["type"] == "CosineAnnealingWarmRestarts":
    params = config["lr_scheduler"]["args"]
    lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=params["T_0"], T_mult=params["T_mult"],
                                                    T_up=params["T_up"], gamma=params["gamma"], eta_max=params["eta_max"])
elif config["lr_scheduler"]["type"] == "GradualWarmupScheduler":
    params = config["lr_scheduler"]["args"]
    scheduler_steplr_args = dict(params["after_scheduler"]["args"])
    scheduler_steplr = getattr(torch.optim.lr_scheduler, params["after_scheduler"]["type"])(optimizer, **scheduler_steplr_args)
    lr_scheduler = GradualWarmupScheduler(optimizer, multiplier=params["multiplier"], total_epoch=params["total_epoch"], after_scheduler=scheduler_steplr)
else:
    lr_scheduler = config.init_obj('lr_scheduler', torch.optim.lr_scheduler, optimizer)

In [13]:
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.valid_data_loader

Loading data...


# train

In [14]:
trainer = Trainer_beat_aligned_data(model, criterion, metrics, optimizer,
                          config=config,
                          data_loader=data_loader,
                          valid_data_loader=valid_data_loader,
                          lr_scheduler=lr_scheduler)

In [15]:
# for batch_idx, ([data, info], target, class_weights) in enumerate(trainer.data_loader):
#     break

In [16]:
# data, target, class_weights, info = data.to(device=trainer.device, dtype=torch.float), target.to(trainer.device, dtype=torch.float), \
#                                                 class_weights.to(trainer.device, dtype=torch.float), info.to(trainer.device, dtype=torch.float)

In [17]:
# output = trainer.model(data, info)

In [18]:
# trainer.criterion(output[:, trainer.indices], target[:, trainer.indices]) * class_weights[:, trainer.indices]

In [19]:
trainer.train()

Train Epoch: 1 [0/319902 (0%)] Loss: 0.747285, 1 batch cost time 0.70
Train Epoch: 1 [512/319902 (0%)] Loss: 0.076057, 1 batch cost time 0.51
Train Epoch: 1 [1024/319902 (0%)] Loss: 0.199569, 1 batch cost time 0.51
Train Epoch: 1 [1536/319902 (0%)] Loss: 0.102687, 1 batch cost time 0.51
Train Epoch: 1 [2048/319902 (1%)] Loss: 0.037557, 1 batch cost time 0.51
Train Epoch: 1 [2560/319902 (1%)] Loss: 0.085971, 1 batch cost time 0.51
Train Epoch: 1 [3072/319902 (1%)] Loss: 0.096871, 1 batch cost time 0.51
Train Epoch: 1 [3584/319902 (1%)] Loss: 0.053256, 1 batch cost time 0.51
Train Epoch: 1 [4096/319902 (1%)] Loss: 0.066219, 1 batch cost time 0.51
Train Epoch: 1 [4608/319902 (1%)] Loss: 0.119105, 1 batch cost time 0.51
Train Epoch: 1 [5120/319902 (2%)] Loss: 0.088440, 1 batch cost time 0.51
Train Epoch: 1 [5632/319902 (2%)] Loss: 0.105053, 1 batch cost time 0.51
Train Epoch: 1 [6144/319902 (2%)] Loss: 0.131343, 1 batch cost time 0.51
Train Epoch: 1 [6656/319902 (2%)] Loss: 0.112705, 1 bat

KeyboardInterrupt: 

# eval

In [20]:
trainer._save_checkpoint(0, save_best=True)

Saving current best: model_best.pth ...


In [21]:
evaluater = Evaluater_beat_aligned_data(model, criterion, metrics,
                          config=config, data_loader = data_loader)
# evaluater = Evaluater_beat_aligned_data(model, criterion, metrics,
#                           config=config, data_loader = data_loader, checkpoint_dir='output/saved/models/beat_aligned_swin_transformer/0622_145002')

Loading checkpoint: output/saved/models/beat_aligned_swin_transformer_CODE/0803_005542/model_best.pth ...


Checkpoint loaded from epoch 1


In [22]:
evaluater.evaluate()

192it [04:03,  1.27s/it]
185it [01:11,  2.59it/s]


In [23]:
# '{}/metrics.json'.format(str(evaluater.checkpoint_dir))

'output/saved/models/beat_aligned_swin_transformer_CODE/0803_004533/metrics.json'

In [31]:
# pathways = str(evaluater.checkpoint_dir).split('/')
# pathways[2] = 'results'
# result_dir = '/'.join(pathways)
# '{}/metrics.json'.format(str(result_dir))

'output/saved/results/beat_aligned_swin_transformer_CODE/0803_004533/metrics.json'

In [69]:
# import json

# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [23]:
# def find_best_thresholds(predictions, true_labels_dict, thresholds):
#     num_classes = len(predictions[0])
#     best_thresholds = [0.5] * num_classes
#     best_f1s = [0.0] * num_classes

#     for class_idx in (range(num_classes)):
#         for thresh in thresholds:
#             f1 = f1_score(
#                 true_labels_dict[class_idx],
#                 predictions[thresh][class_idx],
#                 zero_division=0,
#             )

#             if f1 > best_f1s[class_idx]:
#                 best_f1s[class_idx] = f1
#                 best_thresholds[class_idx] = thresh
    
#     return best_f1s, best_thresholds

In [24]:
# def metrics_table(all_binary_results, all_true_labels):
#     accuracy_scores = []
#     precision_scores = []
#     recall_scores = []
#     f1_scores = []

#     num_classes = all_binary_results.shape[-1]
#     for class_idx in range(num_classes):
#         class_binary_results = all_binary_results[:, class_idx]
#         class_true_labels = all_true_labels[:, class_idx]

#         accuracy = accuracy_score(class_true_labels, class_binary_results)
#         precision = precision_score(class_true_labels, class_binary_results, zero_division=0)
#         recall = recall_score(class_true_labels, class_binary_results, zero_division=0)
#         f1 = f1_score(class_true_labels, class_binary_results, zero_division=0)

#         accuracy_scores.append(accuracy)
#         precision_scores.append(precision)
#         recall_scores.append(recall)
#         f1_scores.append(f1)

#     metrics_dict = {
#         "Accuracy": accuracy_scores,
#         "Precision": precision_scores,
#         "Recall": recall_scores,
#         "F1 Score": f1_scores,
#     }

#     return metrics_dict

In [25]:
# num_classes = 6
# thresholds = np.arange(0, 1.01, 0.01)
# predictions = {thresh: [[] for _ in range(num_classes)] for thresh in thresholds}
# true_labels_dict = [[] for _ in range(num_classes)]

In [26]:
# evaluater.model.eval()
# with torch.no_grad():
#     for batch_idx, ([data, info], target, class_weights) in tqdm(enumerate(evaluater.data_loader.valid_data_loader)):
#         data, target, class_weights, info = data.to(device=evaluater.device, dtype=torch.float), target.to(evaluater.device, dtype=torch.float), \
#                                         class_weights.to(evaluater.device, dtype=torch.float), info.to(evaluater.device, dtype=torch.float)
#         output = evaluater.model(data, info)

#         output_logit = evaluater.sigmoid(output)
#         output_logit = evaluater._to_np(output_logit)
#         target = evaluater._to_np(target)

#         for class_idx in range(num_classes):
#             for thresh in thresholds:
#                 predicted_binary = (output_logit[:, class_idx] >= thresh)
#                 predictions[thresh][class_idx].extend(
#                     predicted_binary
#                 )
#             true_labels_dict[class_idx].extend(
#                 target[:, class_idx]
#             )
#         break

0it [00:00, ?it/s]


In [27]:
# best_f1s, best_thresholds = find_best_thresholds(predictions, true_labels_dict, thresholds)

In [57]:
# all_binary_results = []
# all_true_labels = []

In [58]:
# evaluater.model.eval()
# with torch.no_grad():
#     for batch_idx, ([data, info], target, class_weights) in tqdm(enumerate(evaluater.data_loader.test_data_loader)):
#         data, target, class_weights, info = data.to(device=evaluater.device, dtype=torch.float), target.to(evaluater.device, dtype=torch.float), \
#                                         class_weights.to(evaluater.device, dtype=torch.float), info.to(evaluater.device, dtype=torch.float)
#         output = evaluater.model(data, info)

#         output_logit = evaluater.sigmoid(output)
#         output_logit = evaluater._to_np(output_logit)
#         target = evaluater._to_np(target)

#         binary_result = np.zeros_like(output_logit)
#         for i in range(len(best_thresholds)):
#             binary_result[:, i] = (
#                 output_logit[:, i] >= best_thresholds[i]
#             )
        
#         all_binary_results.append(binary_result)
#         all_true_labels.append(target)
#         break

# all_binary_results = np.array(all_binary_results)[0, :, :]
# all_true_labels = np.array(all_true_labels)[0, :, :]

0it [00:00, ?it/s]


In [77]:
# metrics_dict = metrics_table(all_binary_results, all_true_labels)
# metrics_dict

{'Accuracy': [0.0, 0.03125, 0.984375, 0.75, 0.125, 0.125],
 'Precision': [0.0,
  0.03125,
  0.0,
  0.0,
  0.017543859649122806,
  0.017543859649122806],
 'Recall': [0.0, 1.0, 0.0, 0.0, 1.0, 1.0],
 'F1 Score': [0.0,
  0.06060606060606061,
  0.0,
  0.0,
  0.034482758620689655,
  0.034482758620689655]}

In [73]:
# '{}/metrics.json'.format(str(evaluater.checkpoint_dir))

'output/saved/models/beat_aligned_swin_transformer_CODE/0803_003053/metrics.json'

In [78]:
# with open('{}/metrics.json'.format(str(evaluater.checkpoint_dir)), 'w') as f:
#     json.dump(metrics_dict, f, indent = 2)